In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
import json
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path='/content/drive/MyDrive/Induction_Bot'

In [ ]:
f=open(path+'/classes.json')
data=json.load(f)

In [ ]:
data

In [ ]:
training=[]
output=[]
for intent in data['intents']:
  count=0
  for k in intent:
    for v in intent.get(k):
      for val in intent[k].get(v):
        training.append(val)
        output.append(count)
    count+=1

In [ ]:
training

In [ ]:
output

In [ ]:
import pandas as pd
df=pd.DataFrame({'sentences':training,'class':output})
df.head(34)

In [ ]:
df.to_csv(path+'/sentences.csv')

In [ ]:
df.tail()

In [ ]:
df=df.sample(frac=1)
df


In [ ]:
import pandas as pd
# df=pd.read_csv(path+'/sentence_shuff.csv')
df.to_csv(path+'/shuffled_sent_2.csv')

In [ ]:
data_list=df['sentences'][:]
data_list=list(data_list)

In [ ]:
data_list

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
outputs=list(df['class'])
outputs

In [ ]:
lower_sent=[]
for sen in data_list:
  lower_sent.append(sen.lower())
lower_sent

In [ ]:
from string import punctuation

In [ ]:
newline_removed=[]
for sen in lower_sent:
  newline_removed.append(sen.replace('\n',' '))
newline_removed

In [ ]:
punc_removed=[]
for sen in newline_removed:
  for i in sen:
    if i in punctuation:
      sen=sen.replace(i,'')
  punc_removed.append(sen)
punc_removed

In [ ]:
from nltk.corpus import stopwords
nltk.download('wordnet')

In [ ]:
t='  hello dishant  hi be'
print(t.split())

In [ ]:
stopwords_removed=[]
for sen in punc_removed:
  current=[]
  for word in sen.split():
    if word not in stopwords.words('english'):
      current.append(word)
  stopwords_removed.append(current)
stopwords_removed

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import SnowballStemmer
stemmer=SnowballStemmer('english')
vectorizer=TfidfVectorizer()

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()
lemm_sen=[]
for sen in stopwords_removed:
  current=[]
  for word in sen:
    current.append(lemmatizer.lemmatize(word))
  lemm_sen.append(current)
lemm_sen

In [ ]:
stemmed_sent=[]
for sen in lemm_sen:
  current=[]
  for word in sen:
    current.append(stemmer.stem(word))
  stemmed_sent.append(current)
stemmed_sent

In [ ]:
complete_sent=[]
for sen in stemmed_sent:
  current=''
  for word in sen:
    current=current+word+' '
  complete_sent.append(current)
complete_sent

In [ ]:
X=vectorizer.fit_transform(complete_sent)
print(vectorizer.get_feature_names())

In [ ]:
X.shape

In [ ]:
feature_indices=vectorizer.get_feature_names()
feature_indices

In [ ]:
feature_indices_dict={}
for ind, word in enumerate(feature_indices):
  feature_indices_dict[word]=ind
feature_indices_dict

In [ ]:
# feature_indices_dict={}

import pickle
with open(path+'/feature_indices_3.txt','wb') as f:
  pickle.dump(feature_indices_dict,f)

In [ ]:
tfidf_values=X.toarray()
tfidf_values

In [ ]:
tfidf_values[1][feature_indices_dict['mascot']]

In [ ]:
with open(path+'/tfidf_values_3.txt','wb') as f:
  pickle.dump(tfidf_values,f)

In [ ]:
!pip install tflearn

In [ ]:
import tflearn

In [ ]:
X.shape[1]

In [ ]:
from tensorflow.keras.regularizers import l2

In [ ]:
model=models.Sequential()
model.add(layers.Dense(100,input_dim=X.shape[1],activation='relu'))
model.add(layers.Dense(100,activation='relu'))
model.add(layers.Dense(50,activation='relu'))
model.add(layers.Dense(7,activation='softmax'))
model.summary()

In [ ]:
tfidf_values.shape

In [ ]:

output_np=np.array(outputs)
output_np.shape

In [ ]:
training_tensor=tf.convert_to_tensor(tfidf_values)
testing_tensor=tf.convert_to_tensor(output_np)

In [ ]:
testing_tensor

In [ ]:
from tensorflow.keras.optimizers import SGD

In [ ]:
sgd=SGD()

In [ ]:
model.compile(optimizer=sgd,loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics='accuracy')
history=model.fit(training_tensor,testing_tensor,epochs=1000)

In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics='accuracy')
history=model.fit(training_tensor,testing_tensor,epochs=50)

In [ ]:
model.save(path+'/Induction_Chatbot_6.model')

In [ ]:
text=input()
predict(text)

In [ ]:
model.weights

In [ ]:
t=np.array([[1.5,2.5,3.5]])

In [ ]:
l=[1,2,3,4,5]

In [ ]:
l.insert(0,5)
l

In [ ]:
answers=[[],[],[],[],[],[],[]]
for intent in data['intents']:
  for k, response in zip(intent.keys(),intent.values()):
    print(k)
    for r in response['responses']:
      if k=='greetings':
        answers[0].append(r)
      elif k=='overview':
        answers[1].append(r)
      elif k=='conduct':
        answers[2].append(r)
      elif k=='academics':
        answers[3].append(r)
      elif k=='resources':
        answers[4].append(r)
      elif k=='activities':
        answers[5].append(r)
      elif k=='goodbye':
        answers[6].append(r)

In [ ]:
answers

In [ ]:
with open(path+'/answers_3.txt','wb') as f:
  pickle.dump(answers,f)

In [ ]:
answers[0]

In [ ]:
sentences=answers[6]
lower_sent=[]
for sent in sentences:
  sent=sent.lower()
  lower_sent.append(sent)
punct_sent=[]
for comm in lower_sent:
  for w in comm:
    if w in punctuation:
      comm=comm.replace(w,'')
  punct_sent.append(comm)
comm_words=[]
for comm in punct_sent:
  comm_words.append(comm.split())
cleaned_text=[]
for comm in comm_words:
  words=[]
  for w in comm:
    if w not in set(stopwords.words('english')):
      words.append(w)
  cleaned_text.append(words)
lemmatizer=WordNetLemmatizer()
lemm_text=[]
for text in cleaned_text:
  words=[]
  for w in text:
    words.append(lemmatizer.lemmatize(w))
  lemm_text.append(words)
stemmer=SnowballStemmer('english')
stem_text=[]
for text in lemm_text:
  words=''
  for w in text:
    words=words+stemmer.stem(w)+' '
  stem_text.append(words)
print(stem_text)

In [ ]:
# bot_responses=[]
bot_responses.extend([stem_text])

In [ ]:
bot_responses

In [ ]:
with open(path+'/bot_responses_3.txt','wb') as f:
  pickle.dump(bot_responses,f)

In [ ]:
bot_responses[0]

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers,models
from string import punctuation
import pickle
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
print(nltk.download('stopwords'))
print(nltk.download('wordnet'))
import numpy as np
model=models.load_model(path+'/Induction_Chatbot_6.model')
feature_indices_dict={}
with open(path+'/feature_indices_3.txt','rb') as f:
  feature_indices_dict=pickle.load(f)
answers=[]
with open(path+'/answers_3.txt','rb') as f:
  answers=pickle.load(f)
bot_responses=[]
with open(path+'/bot_responses_3.txt','rb') as f:
  bot_responses=pickle.load(f)

In [ ]:
def predict(text):
  sentences=[text]
  # print(sentences)
  lower_sent=[]
  for sent in sentences:
    sent=sent.lower()
    lower_sent.append(sent)
  punct_sent=[]
  for comm in lower_sent:
    for w in comm:
      if w in punctuation:
        comm=comm.replace(w,'')
    punct_sent.append(comm)
  comm_words=[]
  for comm in punct_sent:
    comm_words.append(comm.split())
  cleaned_text=[]
  for comm in comm_words:
    words=[]
    for w in comm:
      if w not in set(stopwords.words('english')):
        words.append(w)
    cleaned_text.append(words)
  lemmatizer=WordNetLemmatizer()
  lemm_text=[]
  for text in cleaned_text:
    words=[]
    for w in text:
      words.append(lemmatizer.lemmatize(w))
    lemm_text.append(words)
  stemmer=SnowballStemmer('english')
  stem_text=[]
  for text in lemm_text:
    words=''
    for w in text:
      words=words+stemmer.stem(w)+' '
    words=words.replace('colleg','')
    words=words.replace('facil','')
    words=words.replace('campus','')
    stem_text.append(words)
  # print(stem_text[0])
  classes={0:'greetings', 1:'overview',2:'conduct', 3:'academics', 4:'resources',5:'activities',6:'goodbye'}
  vectorizer=TfidfVectorizer()
  tfidf_matrix=vectorizer.fit_transform(stem_text)
  X=tfidf_matrix.toarray()
  feature_arr=vectorizer.get_feature_names()
  feature_arr_dict={}
  for ind,name in enumerate(feature_arr):
    feature_arr_dict[name]=ind
  test_tens=[[0]*784]
  for i in feature_arr:
    try:
      test_tens[0][feature_indices_dict[i]]=X[0][feature_arr_dict[i]]
    except:
      pass
  test_tens=tf.convert_to_tensor(np.array(test_tens))
  predictions=model.predict(test_tens)
  index=np.argmax(predictions)
  responses_list=[]
  responses_list=bot_responses[index][:]
  responses_list.insert(0,stem_text[0])
  count_vectorize=CountVectorizer().fit_transform(responses_list)
  vectors=count_vectorize.toarray()
  csim=cosine_similarity(vectors)
  answer_index=np.argmax(csim[0][1:])
  print(answers[index][answer_index])
  return classes[index]

In [ ]:
while True:
  try:
    c=predict(input("You: "))
    print("\n")
    if c=='goodbye':
      break
  except:
    print("I'm good! How may I help you?")
    pass